In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("Yale-LILY/brio-xsum-cased")
model = AutoModelForSeq2SeqLM.from_pretrained("Yale-LILY/brio-xsum-cased").to(device)

In [2]:
device

'cuda'

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('/teamspace/studios/this_studio/NLP_project/openai_reddit_final.csv', sep=';')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83797 entries, 0 to 83796
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    83797 non-null  int64 
 1   user_id       83797 non-null  object
 2   doc_id        83797 non-null  object
 3   user_profile  83797 non-null  object
 4   post/article  83797 non-null  object
 5   summary_text  83797 non-null  object
 6   confidence    83797 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 4.5+ MB


In [6]:
tokenizer, model

(PegasusTokenizerFast(name_or_path='Yale-LILY/brio-xsum-cased', vocab_size=96103, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask_2>', 'additional_special_tokens': ['<mask_1>', '<unk_2>', '<unk_3>', '<unk_4>', '<unk_5>', '<unk_6>', '<unk_7>', '<unk_8>', '<unk_9>', '<unk_10>', '<unk_11>', '<unk_12>', '<unk_13>', '<unk_14>', '<unk_15>', '<unk_16>', '<unk_17>', '<unk_18>', '<unk_19>', '<unk_20>', '<unk_21>', '<unk_22>', '<unk_23>', '<unk_24>', '<unk_25>', '<unk_26>', '<unk_27>', '<unk_28>', '<unk_29>', '<unk_30>', '<unk_31>', '<unk_32>', '<unk_33>', '<unk_34>', '<unk_35>', '<unk_36>', '<unk_37>', '<unk_38>', '<unk_39>', '<unk_40>', '<unk_41>', '<unk_42>', '<unk_43>', '<unk_44>', '<unk_45>', '<unk_46>', '<unk_47>', '<unk_48>', '<unk_49>', '<unk_50>', '<unk_51>', '<unk_52>', '<unk_53>', '<unk_54>', '<unk_55>', '<unk_56>', '<unk_57>', '<unk_58>', '<unk_59>'

In [7]:
data = df[['doc_id', 'post/article']]

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83797 entries, 0 to 83796
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   doc_id        83797 non-null  object
 1   post/article  83797 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


In [9]:
mapp = {}

for idx , row in data.iterrows():
    doc_id = row['doc_id']
    text = row['post/article']

    if doc_id not in mapp:
        mapp[doc_id] = text

In [10]:
len(mapp)

6218

In [11]:
ids = list(mapp.keys())
len(ids)

6218

In [12]:
unique_texts = list(mapp.values())
len(unique_texts)

6218

In [13]:
batch_size = 5

def translate_batch(texts):
    batch = tokenizer(texts, truncation=True, padding='longest', return_tensors="pt").to(device)
    translated = model.generate(**batch)
    tgt_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_texts
    

In [19]:
results = []

In [20]:
src_texts = unique_texts
len(src_texts)

6218

In [21]:
for i in range(0, len(src_texts), batch_size):
    batch_texts = src_texts[i:i + batch_size]
    translated_texts = translate_batch(batch_texts)
    results.extend(translated_texts)


In [22]:
len(results)

6218

In [24]:
gen_summaries = results

In [25]:
mapp = dict(zip(ids, gen_summaries))

In [28]:
len(mapp)

6218

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83797 entries, 0 to 83796
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    83797 non-null  int64 
 1   user_id       83797 non-null  object
 2   doc_id        83797 non-null  object
 3   user_profile  83797 non-null  object
 4   post/article  83797 non-null  object
 5   summary_text  83797 non-null  object
 6   confidence    83797 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 4.5+ MB


In [30]:
df['brio_model_summary'] = df['doc_id'].map(mapp)

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83797 entries, 0 to 83796
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          83797 non-null  int64 
 1   user_id             83797 non-null  object
 2   doc_id              83797 non-null  object
 3   user_profile        83797 non-null  object
 4   post/article        83797 non-null  object
 5   summary_text        83797 non-null  object
 6   confidence          83797 non-null  int64 
 7   brio_model_summary  83797 non-null  object
dtypes: int64(2), object(6)
memory usage: 5.1+ MB


In [32]:
df.to_csv('openai_reddit_final_brio.csv', sep=';')